### Train and save an ensemble of models
Generate model ensemble based on bootstrapping samples

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import time
import random
from sklearn.model_selection import StratifiedGroupKFold
import xgboost as xgb
import numpy as np
import xarray as xr
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster, wait
from dask_ml.preprocessing import DummyEncoder

# set temperary directory for data spill to scratch, otherwise by default data will spill to the home directory
import dask
dask.config.set({'temporary_directory': '/global/scratch/users/yanghuikang/upscale/data/temp'})

import sys
sys.path.insert(0, '../utility')
import model_evaluation as me
import model_config as mcf

load model_evaluation


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
cluster = LocalCluster(ip='*') 
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.0.7.89:8787/status,
Dashboard: http://10.0.7.89:8787/status,Workers: 8
Total threads: 32,Total memory: 89.84 GiB
Status: running,Using processes: True
Comm: tcp://10.0.7.89:40453,Workers: 8
Dashboard: http://10.0.7.89:8787/status,Total threads: 32
Started: Just now,Total memory: 89.84 GiB
Comm: tcp://10.0.7.89:33706,Total threads: 4
Dashboard: http://10.0.7.89:45116/status,Memory: 11.23 GiB
Nanny: tcp://10.0.7.89:41466,


In [4]:
# read training data, need to use dask array for xgb.dask to work
data_path = '/global/scratch/users/yanghuikang/upscale/data/site/fulldata/'
train_st = pd.read_csv(data_path+'data_full_oneflux_features_shortterm_clean_v1_2.csv')
train_lt = pd.read_csv(data_path+'data_full_oneflux_features_longterm_clean_v1_2.csv')


In [5]:
# create a list of model configurations
st_config = mcf.param_st_dict['xgboost']

model_config_ST_Baseline_NT = {'target':'GPP_NT_VUT_REF',
                               'features':mcf.ft_setup_dict['ST_Baseline'],
                               'name':'ST_Baseline_NT',
                               'hyper_param': st_config}

model_config_ST_Baseline_DT = {'target':'GPP_DT_VUT_REF',
                               'features':mcf.ft_setup_dict['ST_Baseline'],
                               'name':'ST_Baseline_DT',
                               'hyper_param': st_config}

model_config_ST_CFE_NT = {'target':'GPP_NT_VUT_REF',
                          'features':mcf.ft_setup_dict['ST_CFE'],
                          'name':'ST_CFE_NT',
                          'hyper_param': st_config}

model_config_ST_CFE_DT = {'target':'GPP_DT_VUT_REF',
                          'features':mcf.ft_setup_dict['ST_CFE'],
                          'name':'ST_CFE_DT',
                          'hyper_param': st_config}

"""
lt_config = mcf.param_lt_dict['xgboost']
model_config_LT_Baseline_NT = {'target':'GPP_NT_VUT_REF',
                               'features':mcf.ft_setup_dict['LT_Baseline'],
                               'name':'LT_Baseline_NT',
                               'hyper_param': lt_config}

model_config_LT_Baseline_DT = {'target':'GPP_DT_VUT_REF',
                               'features':mcf.ft_setup_dict['LT_Baseline'],
                               'name':'LT_Baseline_DT',
                               'hyper_param': lt_config}

model_config_LT_CFE_NT = {'target':'GPP_NT_VUT_REF',
                          'features':mcf.ft_setup_dict['LT_CFE'],
                          'name':'LT_CFE_NT',
                          'hyper_param': lt_config}

model_config_LT_CFE_DT = {'target':'GPP_DT_VUT_REF',
                          'features':mcf.ft_setup_dict['LT_CFE'],
                          'name':'LT_CFE_DT',
                          'hyper_param': lt_config}
"""

config_list = [model_config_ST_Baseline_NT, model_config_ST_CFE_NT, model_config_ST_Baseline_DT, model_config_ST_CFE_DT]

In [6]:
# generate a list of random seed
random.seed(mcf.RANDOM_STATE)
random_list = random.sample(range(1,1000),30)

In [7]:
# create a random selection of 80% of data, cover all PFTs, and data from one site will not be split
def getSplit(data, random_state):
    
    n_fold = 5
    group = 'SITE_ID'
    stratify_class = 'MODIS_IGBP'
    group_kfold = StratifiedGroupKFold(n_splits=n_fold,shuffle=True,random_state=random_state)
    train_split = [[tr,te] for tr, te in group_kfold.split(data,data[stratify_class],data[group])]
    train_index = train_split[0][0]
    data_sample = data.iloc[train_index,]
    
    data_sample = data_sample.sample(frac=1,ignore_index=True) # re-index after shuffle
    
    return data_sample

In [8]:
# create a bootstrapping sample of sites
def getBootstrap(data, random_state):

    sites_list = data.groupby(['SITE_ID','MODIS_PFT','SITE_IGBP']).agg({'month':'count'}).reset_index()
    sample = sites_list.sample(n=sites_list.shape[0],replace=True, random_state = random_state).reset_index()
    sample.loc[:,'uid'] = pd.Series(range(sample.shape[0]))
    data_sample = sample.groupby(['uid','SITE_ID']).apply(lambda x: data[data['SITE_ID']==x.iloc[0]['SITE_ID']])
    data_sample = data_sample.drop(columns=['SITE_ID']).reset_index()
    
    data_sample = data_sample.sample(frac=1,ignore_index=True) # shuffle the data, re-index after shuffle

    print('No. of unique site: {}'.format(len(data_sample['SITE_ID'].unique())))
    print('No. of site months: {}'.format(data_sample.shape[0]))
    print('Unique MODIS_PFT: {}'.format(data_sample['MODIS_PFT'].unique()))

    return data_sample

# for random_state in random_list:
#     train = getBootstrap(train_st, random_state)

In [9]:
def saveModel(data, config, outname):
    """
    Train and save xgboost model
    """
    
    start = time.time()

    ft_list = config['features']
    target_col = config['target']
    num_boost_round = config['hyper_param']['n_estimators']
    gamma = config['hyper_param']['gamma']
    learning_rate = config['hyper_param']['learning_rate']
    colsample_bytree = config['hyper_param']['colsample_bytree']

    # convert to Dask matrix
    dtrain = xgb.dask.DaskDMatrix(client,dd.from_pandas(data[ft_list],npartitions=1),
                                  dd.from_pandas(data[target_col],npartitions=1),feature_names=ft_list)
    # print(dtrain.num_col())
    # print(dtrain.feature_names)
    
    # xgboost trianing
    xgb_params = {'colsample_bytree': colsample_bytree, 'gamma': gamma, 'learning_rate': learning_rate, 'verbosity':0}
    xgb_output = xgb.dask.train(client, xgb_params, dtrain, num_boost_round=num_boost_round, evals=[(dtrain,'train')],
                               verbose_eval=False)
    
    pred = xgb.dask.predict(client,xgb_output['booster'],dtrain)
    train_pred = data[['SITE_ID','year','month',target_col]].copy()
    train_pred.loc[:,'pred'] = pred.compute()
    
    # training accuracy
    accu = me.getMetrics(train_pred['pred'],train_pred[target_col],'')
    
    # save model
    # print(accu)
    print('training accu.: {:.2f}'.format(accu['_rmse']))
    print('saving to ...'+outname,flush=True)
    xgb_output['booster'].save_model(outname)
    print('time lapse ... {:d} seconds'.format(int(time.time()-start)))
    

In [10]:
%%time
data_train = getBootstrap(train_st,1)
config = model_config_ST_Baseline_NT
saveModel(data_train, config, 'test')

No. of unique site: 145
No. of site months: 19062
Unique MODIS_PFT: ['CRO' 'SA' 'MF' 'Other' 'GRA' 'DBF' 'ENF' 'EBF' 'SH']


/global/scratch/users/yanghuikang/conda/env/ml-new/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[10:50:49] task [xgboost.dask]:tcp://10.0.7.89:33706 got new rank 0
/global/scratch/users/yanghuikang/conda/env/ml-new/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/global/scratch/users/yanghuikang/conda/env/ml-new/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/global/scratch/users/yanghuikang/c

training accu.: 1.34
saving to ...test
time lapse ... 12 seconds
CPU times: user 1.68 s, sys: 1.14 s, total: 2.81 s
Wall time: 13.2 s


In [11]:
%%time
out_path = '/global/home/users/yanghuikang/projects/upscale/saved_models/ensemble_v1_2/'

for config in config_list:
    model_name = config['name']
    print('\n'+model_name)
    
    # create 30 ensemble models
    for i,value in enumerate(random_list):
        print('ensemble {}'.format(i))
        outname = out_path+'xgboost_'+model_name+'_ensemble_'+str(i)+'.json'
        data_train = getBootstrap(train_st,value)
        saveModel(data_train, config, outname)
        
        # break
    
    # create a reference model (all samples used for training)
    print('reference model')
    outname = out_path+'xgboost_'+model_name+'_reference.json'
    saveModel(train_st, config, outname)
    
    # break


ST_Baseline_NT
ensemble 0
No. of unique site: 139
No. of site months: 16997
Unique MODIS_PFT: ['CRO' 'MF' 'SA' 'Other' 'GRA' 'ENF' 'DBF' 'SH' 'EBF']


[10:51:46] task [xgboost.dask]:tcp://10.0.7.89:33706 got new rank 0


training accu.: 1.29
saving to .../global/home/users/yanghuikang/projects/upscale/saved_models/ensemble_v1_2/xgboost_ST_Baseline_NT_ensemble_0.json
time lapse ... 8 seconds
ensemble 1
No. of unique site: 153
No. of site months: 18143
Unique MODIS_PFT: ['GRA' 'MF' 'CRO' 'SH' 'SA' 'ENF' 'DBF' 'EBF' 'Other']


[10:51:55] task [xgboost.dask]:tcp://10.0.7.89:33706 got new rank 0


training accu.: 1.37
saving to .../global/home/users/yanghuikang/projects/upscale/saved_models/ensemble_v1_2/xgboost_ST_Baseline_NT_ensemble_1.json
time lapse ... 8 seconds
ensemble 2
No. of unique site: 145
No. of site months: 18198
Unique MODIS_PFT: ['CRO' 'SA' 'ENF' 'GRA' 'DBF' 'Other' 'SH' 'MF' 'EBF']


...

training accu.: 1.21
saving to .../global/home/users/yanghuikang/projects/upscale/saved_models/ensemble_v1_2/xgboost_ST_CFE_DT_ensemble_28.json
time lapse ... 9 seconds
ensemble 29
No. of unique site: 147
No. of site months: 16816
Unique MODIS_PFT: ['MF' 'SA' 'EBF' 'GRA' 'ENF' 'Other' 'CRO' 'DBF' 'SH']


[11:11:02] task [xgboost.dask]:tcp://10.0.7.89:33706 got new rank 0


training accu.: 1.29
saving to .../global/home/users/yanghuikang/projects/upscale/saved_models/ensemble_v1_2/xgboost_ST_CFE_DT_ensemble_29.json
time lapse ... 8 seconds
reference model


[11:11:11] task [xgboost.dask]:tcp://10.0.7.89:33706 got new rank 0


training accu.: 1.42
saving to .../global/home/users/yanghuikang/projects/upscale/saved_models/ensemble_v1_2/xgboost_ST_CFE_DT_reference.json
time lapse ... 8 seconds
CPU times: user 3min 1s, sys: 41.8 s, total: 3min 43s
Wall time: 19min 34s
